# SD-TSIA 211
# Optimization for Machine Learning

# Computer lab : Let’s reverse-engineer the data center 

**By Hamza El Arji**

## Imports

In [116]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## PART 2 :  Database and statistical model


In [96]:
# Loading data
data_matrix_train, COP_train, data_matrix_test, COP_test, names = np.load('data_center_data_matrix.npy', allow_pickle=True)

# Constructing matrices for min_w ||A w - b||_2**2
matrix_mean = np.mean(data_matrix_train, axis=0) # Calculate mean of training data matrix along each column
M = data_matrix_train - matrix_mean # Center the training data matrix by subtracting the mean
matrix_std = np.std(M, axis=0) # Calculate standard deviation of centered data matrix along each column
M = M / matrix_std # Normalize the centered data matrix by dividing by standard deviation

# Construct matrix A and vector b for the training set
A = np.hstack([M, np.ones((M.shape[0],1)), -(M.T * COP_train[:,3]).T])
b = COP_train[:,3]

# Constructing matrices for the test set
M_test = (data_matrix_test - matrix_mean) / matrix_std
A_test = np.hstack([M_test, np.ones((M_test.shape[0],1)), -(M_test.T * COP_test[:,3]).T])
b_test = COP_test[:,3]


# Loading raw data
import pandas as pd
data = pd.read_csv('Raw_Dataset_May.csv')

def name_to_subcategory_and_details(col_name):
    if np.isreal(col_name):
        col_name = names[col_name]
    indices = np.nonzero((data['NAME'] == col_name).values)[0]
    if len(indices) > 0:
        subcategory = data['SUBCATEGORY'].iloc[[indices[0]]].values[0]
        details = data['DETAILS'].iloc[[indices[0]]].values[0]
        return subcategory, details
    else:
        print('unknown name')


## PART 3 :  Least squares


**question 3.1**

**To demonstrate that if** \(Aw=b\), **then**
$ y(t) = \frac{w_1^T \tilde{x}(t)  + w_0}{w_2^T \tilde{x}(t)  + 1} $ 

**Let's consider the expression for** $ (Aw)_t $ **and simplify it:**
$ (Aw)_t = \tilde{x}(t)^T w_1 + w_0 - y(t) \times \tilde{x}(t)^T w_2 $ 

**Given that** \(Aw = b\), **we can replace** $(Aw)_t $ **with** $(b_t = y(t))$:
$  y(t) =  \tilde{x}(t)^T w_1 + w_0 - y(t) \times \tilde{x}(t)^T w_2  $

**So, we have:**
$ y(t) = \frac{w_1^T \tilde{x}(t)  + w_0}{w_2^T \tilde{x}(t)  + 1} $ 



**question 3.2**

In [97]:
# Solve the OLS problem
w_optimal, residuals, _, _ = np.linalg.lstsq(A, b, rcond=None)
print("Optimized Parameters:")
print(w_optimal)




Optimized Parameters:
[-0.00927821  0.08309371 -0.03672704 ...  0.01980595 -0.03057174
 -0.01188614]


**question 3.3**

In [98]:
# compute the predicted values and the MSE
predicted_values =A_test @ w_optimal
mse = np.mean((predicted_values - b_test) ** 2)
print("Mean Squared Error:", mse)


Mean Squared Error: 780.8984793523564


**question 3.4**

In [127]:
# Regularization parameter
lambda_val = 100

# Construct the regularized matrices
A_reg = np.vstack([A, np.sqrt(lambda_val) * np.eye(A.shape[1])])
b_reg = np.concatenate([b, np.zeros(A.shape[1])])

# Solve the regularized least squares problem
w_optimal_reg, residuals_reg, _, _ = np.linalg.lstsq(A_reg, b_reg, rcond=None)
print(w_optimal_reg)

# Predict the values using the optimized parameters
predicted_values_reg = np.dot(A_test, w_optimal_reg)

# Calculate the mean squared error for the regularized problem
mse_reg = np.mean((predicted_values_reg - b_test) ** 2)

# Print the MSE for both problems
print("Unregularized Mean Squared Error:", mse)
print("Regularized Mean Squared Error:", mse_reg)

[-0.01238313  0.05780406 -0.00127775 ...  0.01590558 -0.03567897
  0.0131232 ]
Unregularized Mean Squared Error: 780.8984793523564
Regularized Mean Squared Error: 301.0548280940246


**question 3.5**

In [100]:
# Calculate the gradient and the hessian matrice of f1
gradient_f1 = np.dot(A.T, np.dot(A, w_optimal_reg) - b) + lambda_val *w_optimal_reg
hessian_f1 = np.dot(A.T, A) + lambda_val * np.eye(A.shape[1])

# Is the function convex ?
is_convex = np.all(np.linalg.eigvals(hessian_f1) >= 0)
if is_convex:
    print("The function is convex.")
else:
    print("The function is not convex.")

The function is convex.


**question 3.6**

In [118]:
# Initialize variables for gradient descent
w = np.zeros(A.shape[1])
step_size = 0.0000001
convergence_threshold = 1
iterations = 0
positive_infinity = float('inf')
gradient_norm = positive_infinity

# Perform gradient descent
while gradient_norm > convergence_threshold and iterations<110000:
    gradient = 2 * np.dot(A.T, np.dot(A, w) - b) + lambda_val * w
    w -= step_size * gradient
    gradient_norm = np.linalg.norm(gradient)
    iterations += 1

# Print the step size and the number of iterations
print("Step size:", step_size)
print("Number of iterations:", iterations)
print("Gradient norm:", gradient_norm)
print("Gradient optimal:", np.linalg.norm(gradient_f1))




Step size: 1e-07
Number of iterations: 110000
Gradient norm: 5.0435923808453795
Gradient optimal: 1.313600338502328e-10


## PART 4 :  Regularization for a sparse model



**question 4.1**

To formulate the objective function  $ F_2 $ for the Lasso problem, we need to decompose it into two parts: a differentiable part $ f_2 $ and a non-differentiable part $ g_2 $. The proximal operator of $ g_2 $ should be easy to compute.

The Lasso objective function is defined as follows:

$ F_2(w) = \frac{1}{2} \|Aw - b\|_2^2 + \lambda \|w\|_1 $

Now, let's decompose it into $ f_2 $ and $ g_2 $:

$  f_2(w) = \frac{1}{2} \|Aw - b\|_2^2 $   and  $ g_2(w) = \lambda \|w\|_1 $

The proximal operator $ \text{prox}_{g_2}(v) $ is defined as:

$ \text{prox}_{g_2}(v) = \arg\min_w \left( g_2(w) + \frac{1}{2} \|v-w\|_2^2 \right) $

Now, let's calculate the gradient of $ f_2(w) $:

$ \nabla f_2(w) = A^T(Aw - b) $

This is because the gradient of the quadratic term $ \frac{1}{2} \|Aw - b\|_2^2 $ with respect to $ w $ is $ A^T(Aw - b) $.

So, in summary:

$ F_2(w) = f_2(w) + g_2(w) \\ $ 
$ f_2(w) = \frac{1}{2} \|Aw - b\|_2^2 \\ $
$ g_2(w) = \lambda \|w\|_1 \\$
$  \nabla f_2(w) = A^T(Aw - b) \\$

This formulation allows us to apply proximal gradient methods or other optimization algorithms that can handle proximal operators for solving Lasso problems.


**question 4.2**

In [124]:
# Define the objective function
def objective_function(A, b, w, lambda_val):
    return 0.5 * np.linalg.norm(np.dot(A, w) - b)**2 + lambda_val * np.linalg.norm(w, 1)

# Define the gradient of the objective function
def gradient(A, b, w):
    return np.dot(A.T, np.dot(A, w) - b)

# Define the proximal operator for L1 regularization
def proximal_operator(w, lambda_val, step_size):
    return np.sign(w) * np.maximum(np.abs(w) - lambda_val * step_size, 0)

# Set the parameters
lambda_val = 200
step_size = 1e-07
max_iterations = 1000

# Initialize the weight vector
w = np.zeros(A.shape[1])

# Perform proximal gradient descent
for iteration in range(max_iterations):
    # Calculate the gradient
    grad = gradient(A, b, w)
    
    # Update the weight vector using proximal operator
    w = proximal_operator(w - step_size * grad, lambda_val * step_size, step_size)
    
# Print the optimized weight vector
print("Optimized Weight Vector:")
print(w)





Optimized Weight Vector:
[-0.00406567  0.01906334  0.01028991 ...  0.00188562 -0.00609072
 -0.00041177]


**question 4.3**

In [125]:
import time

# Define the objective function
def objective_function(A, b, w, lambda_val):
    return 0.5 * np.linalg.norm(np.dot(A, w) - b)**2 + lambda_val * np.linalg.norm(w, 1)

# Define the gradient of the objective function
def gradient(A, b, w):
    return np.dot(A.T, np.dot(A, w) - b)

# Define the proximal operator for L1 regularization
def proximal_operator(w, lambda_val, step_size):
    return np.sign(w) * np.maximum(np.abs(w) - lambda_val * step_size, 0)

# Set the parameters
lambda_val = 200
max_iterations = 1000

# Initialize the weight vector
w = np.zeros(A.shape[1])

# Perform proximal gradient descent with fixed step size
start_time_fixed = time.time()
for iteration in range(max_iterations):
    # Calculate the gradient
    grad = gradient(A, b, w)
    
    # Update the weight vector using proximal operator
    w = proximal_operator(w - step_size * grad, lambda_val * step_size, step_size)
    
end_time_fixed = time.time()

# Perform proximal gradient descent with line search
w = np.zeros(A.shape[1])
step_size = 1e-07
start_time_line_search = time.time()
for iteration in range(max_iterations):
    # Calculate the gradient
    grad = gradient(A, b, w)
    
    # Perform line search to find the optimal step size
    t = 1.0
    while objective_function(A, b, w - t * grad, lambda_val) > objective_function(A, b, w, lambda_val) - 0.5 * t * np.linalg.norm(grad)**2:
        t *= 0.5
    
    # Update the weight vector using proximal operator with the optimal step size
    w = proximal_operator(w - t * grad, lambda_val * t, t)
    
end_time_line_search = time.time()

# Calculate the execution time for both approaches
execution_time_fixed = end_time_fixed - start_time_fixed
execution_time_line_search = end_time_line_search - start_time_line_search

# Print the execution times
print("Execution Time with Fixed Step Size:", execution_time_fixed)
print("Execution Time with Line Search:", execution_time_line_search)





Execution Time with Fixed Step Size: 1.2745351791381836
Execution Time with Line Search: 87.93779373168945


## PART 6 :  Comparison



**Optimized Weight Vector for l1:** 

**[-0.00406567  0.01906334  0.01028991 ...  0.00188562 -0.00609072 -0.00041177]** 
 
**Optimized Weight Vector for l2:**

 **[-0.01238313  0.05780406 -0.00127775 ...  0.01590558 -0.03567897 0.0131232 ]**



## PART 7 :  Commenting 2 reports



d

